
# Retrieval-Augmented Shopping Assistant - Embeddings Experimentations

In [1]:
# Generate Embeddings from embedding_input

!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 8.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [3]:
# Use the parquet file generated in EDA phase.

import pandas as pd

inScopeMetadata = pd.read_parquet("/kaggle/input/abo-english-metadata-parquet/inScopeMetadata_with_embeddings.parquet")

In [4]:
# check the type before resuming.

type(inScopeMetadata['embedding_vector'].iloc[0])


numpy.ndarray

In [ ]:
#Load model and encode

from sentence_transformers import SentenceTransformer

# Load the model (compact + effective)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate dense embeddings (512-dimensional vectors)
embedding_list = model.encode(
    inScopeMetadata['embedding_input'].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True
)

2025-05-02 03:14:27.189002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746155667.467807      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746155667.550841      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3836 [00:00<?, ?it/s]

In [ ]:
# Attach to the dataframe

import numpy as np

# Save as separate column or matrix
inScopeMetadata['embedding_vector'] = list(embedding_list)

In [ ]:
inScopeMetadata.sample(1)

In [ ]:
import numpy as np

embedding_matrix = np.vstack(embedding_list)

In [ ]:
!pip install faiss-cpu --quiet

In [ ]:
# Create vector index

import faiss

dim = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embedding_matrix)  # 🔥 fast vector search

In [ ]:
np.save("embedding_matrix.npy", embedding_matrix)
#embedding_matrix = np.load("embedding_matrix.npy")
faiss.write_index(index, "faiss_index.index")
# index = faiss.read_index("faiss_index.index")

With above copies, we have:

faiss_index.index → FAISS binary index
inScopeMetadata_with_embeddings.parquet → all metadata + embedding inputs
embedding_matrix.npy → optional fallback